In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import polars as pl
from datetime import timedelta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import gc


In [2]:
# Wczytaj i przetwórz mr_linear
mr_linear = pl.read_csv('C:\\Users\\user\\Documents\\danonki\\model_df_linear.csv', ignore_errors=True)
mr_linear = (
    mr_linear
    .select(['patient_id', 'recording', 'create_date', 'date', 'visit_date', 'time_point', 
             'confidence', 'label', 'chunks_count', 'euthymia', 'depression', 'mania', 'mixed'])
    .rename({"recording": "mr", "confidence": "confidence_linear"})
    .with_columns([
        pl.when(pl.col("mr") == "NA").then(None).otherwise(pl.col("mr")).cast(pl.Int64).alias('mr')
    ])
)

# Wczytaj i przetwórz mr_constant
mr_constant = pl.read_csv('C:\\Users\\user\\Documents\\danonki\\model_df_constant.csv', ignore_errors=True)
mr_constant = (
    mr_constant
    .select([ 'recording', 'confidence'])  # Tylko potrzebne kolumny
    .rename({"recording": "mr", "confidence": "confidence_constant"})
    .with_columns([
        pl.when(pl.col("mr") == "NA").then(None).otherwise(pl.col("mr")).cast(pl.Int64).alias('mr')
    ])
)

# Wczytaj i przetwórz mr_gauss
mr_gauss = pl.read_csv('C:\\Users\\user\\Documents\\danonki\\model_df.csv', ignore_errors=True)
mr_gauss = (
    mr_gauss
    .select([ 'recording', 'confidence'])  # Tylko potrzebne kolumny
    .rename({"recording": "mr", "confidence": "confidence_gauss"})
    .with_columns([
        pl.when(pl.col("mr") == "NA").then(None).otherwise(pl.col("mr")).cast(pl.Int64).alias('mr')
    ])
)

# # Połącz wszystkie ramki danych na podstawie kolumny "mr" i wspólnych atrybutów
mr = (
    mr_linear
    .join(mr_constant, on="mr" )#, how="left")
    .join(mr_gauss, on="mr" ) #, how="left")
)




In [3]:
ramka = mr.group_by(['patient_id','visit_date','label']).agg(pl.col("label").count().alias("count")).sort('patient_id').filter(pl.col('visit_date')!='NA')
ramka.group_by('label').agg(pl.col('count').sum().alias('suma'))

label,suma
str,u32
"""euthymia""",1698
"""depression""",1589
"""mania""",816
"""mixed""",434


In [4]:
gc.collect()

30

## Podział na 5 grup

In [10]:
dane_1472 =  pl.read_csv('C:\\Users\\user\\Documents\\danonki\\vol2\\1472_all.csv' , n_rows = 5000000 , ignore_errors=True ) #12sek

dane_2004 =  pl.read_csv('C:\\Users\\user\\Documents\\danonki\\2004_data.csv' , n_rows = 5000000 , ignore_errors=True ) #11sek
dane_2004 = dane_2004.drop(dane_2004.columns[0]) #jeśli jest 100 kolumn

dane_6139 =  pl.read_csv('C:\\Users\\user\\Documents\\danonki\\vol1\\6139_data.csv' , n_rows = 5000000 , ignore_errors=True ) #8sek
dane_6139 = dane_6139.drop(dane_6139.columns[0]) #jeśli jest 100 kolumn

dane_8281 =  pl.read_csv('C:\\Users\\user\\Documents\\danonki\\vol1\\8281_data.csv' , n_rows = 5000000 , ignore_errors=True ) #8sek
dane_8281 = dane_8281.drop(dane_8281.columns[0]) #jeśli jest 100 kolumn

dane1 = [dane_1472, dane_2004, dane_6139, dane_8281]



KeyboardInterrupt: 

In [5]:
probka =  pl.read_csv('C:\\Users\\user\\Documents\\danonki\\vol2\\1472_all.csv' , n_rows = 50 , ignore_errors=True ) #12sek
kolumny = probka.columns
reference_schema = probka.schema

In [ ]:
def process_dane_data(dane, mr ):
    # Step 1: Perform a left join on the 'mr' column
    dane = dane.join(mr, on='mr', how='left')
    start_col='pcm_LOGenergy_sma' 
    end_col='pcm_fftMag_mfcc_12_'
    # Step 2: Dynamically select columns between start_col and end_col, plus additional specified columns
    all_columns = dane.columns
    start_idx = all_columns.index(start_col)
    end_idx = all_columns.index(end_col)
    
    # Select specific columns along with the dynamically sliced columns
    dane = dane.select(
        ['mr', 'patient_id', 'chunk_number', 'frame_nr', 'create_date', 'date', 'visit_date', 'time_point',  'label', 'chunks_count', 'euthymia', 'depression', 'mania', 'mixed','confidence_gauss', 'confidence_linear' ,'confidence_constant' ] 
        + all_columns[start_idx:end_idx + 1]
    )
    
    # Step 3: Filter rows where 'time_point' is in the specified range
    dane = dane.filter(pl.col('time_point').is_in(['-7', '-6', '-5', '-4', '-3', '-2', '-1', '0', '1', '2']))
    
    return dane
 
# Funkcja do rzutowania kolumn na typy zgodne ze schematem referencyjnym
def cast_to_reference_schema(df, reference_schema):
    return df.with_columns([
        pl.col(col_name).cast(reference_schema[col_name]) 
        for col_name in df.columns if col_name in reference_schema  # Rzutowanie tylko istniejących kolumn
    ])

In [ ]:

# Utwórz próbkę danych, aby ustalić `reference_schema`
probka = pl.read_csv('C:\\Users\\user\\Documents\\danonki\\vol2\\1472_all.csv', n_rows=50, ignore_errors=True)
kolumny = probka.columns
reference_schema = probka.schema


mr = mr.with_columns([pl.col("mr").cast(pl.Int64)])
mr = mr.lazy()

In [34]:
file_paths = [
    # Pliki z katalogu vol1
    # 'C:\\Users\\user\\Documents\\danonki\\vol1\\0681_data.csv',
    # 'C:\\Users\\user\\Documents\\danonki\\vol1\\1153_data.csv',
    # 'C:\\Users\\user\\Documents\\danonki\\vol1\\2500_data.csv',
    # 'C:\\Users\\user\\Documents\\danonki\\vol1\\4614_data.csv',
    # 'C:\\Users\\user\\Documents\\danonki\\vol1\\4953_data.csv',
    'C:\\Users\\user\\Documents\\danonki\\vol1\\5656_data.csv',
    'C:\\Users\\user\\Documents\\danonki\\vol1\\6139_data.csv',
    #'C:\\Users\\user\\Documents\\danonki\\vol1\\6241_data.csv',
    #'C:\\Users\\user\\Documents\\danonki\\vol1\\6601_data.csv',
    #'C:\\Users\\user\\Documents\\danonki\\vol1\\6754_data.csv',
    #'C:\\Users\\user\\Documents\\danonki\\vol1\\7297_data.csv',
    #'C:\\Users\\user\\Documents\\danonki\\vol1\\7379_data.csv',
    #'C:\\Users\\user\\Documents\\danonki\\vol1\\8193_data.csv',
    #'C:\\Users\\user\\Documents\\danonki\\vol1\\8281_data.csv',
    #'C:\\Users\\user\\Documents\\danonki\\vol1\\8779_data.csv',
    #'C:\\Users\\user\\Documents\\danonki\\vol1\\8866_data.csv',
    #'C:\\Users\\user\\Documents\\danonki\\vol1\\9813_data.csv',
    #'C:\\Users\\user\\Documents\\danonki\\vol1\\9829_data.csv',
    
    
    # Pliki z katalogu vol2
    #'C:\\Users\\user\\Documents\\danonki\\vol2\\1472_all.csv',
    #'C:\\Users\\user\\Documents\\danonki\\vol2\\1981_all.csv',
    #'C:\\Users\\user\\Documents\\danonki\\vol2\\2582_all.csv',
    #'C:\\Users\\user\\Documents\\danonki\\vol2\\6701_all.csv',
    #'C:\\Users\\user\\Documents\\danonki\\vol2\\8560_all.csv',

    # Pliki spoza katalogów vol1 i vol2
    #'C:\\Users\\user\\Documents\\danonki\\2004_data.csv'
]

## Przetwarzanie w grupach 

In [34]:
def process_dane_data(dane, mr ):
    # Step 1: Perform a left join on the 'mr' column
    dane = dane.join(mr, on='mr', how='left')
    start_col='pcm_LOGenergy_sma' 
    end_col='pcm_fftMag_mfcc_12_'
    # Step 2: Dynamically select columns between start_col and end_col, plus additional specified columns
    all_columns = dane.columns
    start_idx = all_columns.index(start_col)
    end_idx = all_columns.index(end_col)
    
    # Select specific columns along with the dynamically sliced columns
    dane = dane.select(
        ['mr', 'patient_id', 'chunk_number', 'frame_nr', 'create_date', 'date', 'visit_date', 'time_point', 
         'confidence', 'label', 'chunks_count', 'euthymia', 'depression', 'mania', 'mixed'] 
        + all_columns[start_idx:end_idx + 1]
    )
    
    # Step 3: Filter rows where 'time_point' is in the specified range
    dane = dane.filter(pl.col('time_point').is_in(['-7', '-6', '-5', '-4', '-3', '-2', '-1', '0', '1', '2']))
    
    return dane


In [60]:

# Funkcja do rzutowania kolumn zgodnie z `reference_schema`
def cast_to_reference_schema(df, reference_schema):
    return df.with_columns([
        pl.col(col_name).cast(reference_schema[col_name]) 
        for col_name in df.columns if col_name in reference_schema
    ])

def cast_to_reference_schema_special(df, reference_schema):
    return df.with_columns([
        # Jeśli docelowy typ to Int64, wykonaj najpierw cast na Float64, a potem na Int64
        pl.col(col_name).cast(pl.Float64).cast(pl.Int64) if reference_schema[col_name] == pl.Int64 
        else pl.col(col_name).cast(reference_schema[col_name])  # Dla pozostałych kolumn wykonaj zwykłe rzutowanie
        for col_name in df.columns if col_name in reference_schema
    ])

# Przetwarzanie plików
for i, path in enumerate(file_paths):
    print(f"Przetwarzanie pliku {i + 1}/{len(file_paths)}: {path}")

    # Lazy load ramki z rzutowaniem
    dane_lazy = (
        pl.scan_csv(path, ignore_errors=True)
        .select(kolumny)
        .with_columns([
            pl.col("mr").cast(pl.Int64),  # Upewnij się, że `mr` jest spójny
            pl.col("chunk_number").cast(pl.Float64).cast(pl.Int64) ,
            #pl.col('frame_nr').cast(pl.Float64).cast(pl.Int64) 
        ])
        .filter(pl.col('chunk_number') == 0)
    )


    # Rzutowanie kolumn zgodnie z `reference_schema` - zgodnie z kodem z zapytania
    if os.path.basename(path) in ["6139_data.csv", "5656_data.csv"]:  # Warunek na pliki 6139 i 5656
        dane_lazy = cast_to_reference_schema_special(dane_lazy, reference_schema)

    # Przetwarzanie danych
    data_lazy = process_dane_data(dane=dane_lazy, mr=mr)
    
    # Zapis wyników do pliku
    output_path = f'./danonki/{os.path.splitext(os.path.basename(path))[0]}_confidence.csv'
    
    try:
        data_lazy.sink_csv(output_path)
        print(f'Zapisano plik: {output_path}')
    except Exception as e:
        print(f"Błąd podczas zapisu pliku {output_path}: {e}")

Przetwarzanie pliku 1/2: C:\Users\user\Documents\danonki\vol1\5656_data.csv


C:\Users\user\AppData\Local\Temp\ipykernel_14828\2621394214.py:13: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  for col_name in df.columns if col_name in reference_schema
C:\Users\user\AppData\Local\Temp\ipykernel_14828\2651908419.py:7: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  all_columns = dane.columns


Zapisano plik: ./danonki/5656_data_confidence.csv
Przetwarzanie pliku 2/2: C:\Users\user\Documents\danonki\vol1\6139_data.csv
Zapisano plik: ./danonki/6139_data_confidence.csv
